In [23]:
import pandas as pd
import pathlib
import sqlite3
import numpy as np

import plotly.express as px

In [6]:
Operations = {
    ".csv": pd.read_csv,
    ".xlsx": pd.read_excel,
    ".json": pd.read_json,
    ".db": lambda p: pd.read_sql('SELECT * from all_noisy_table', sqlite3.connect(p))
}

def read_file(path: str) -> pd.DataFrame:
    file = pathlib.Path(path)
    try:
        return Operations[file.suffix](path).fillna(value=np.nan).replace("unknown", np.nan)
    except KeyError:
        raise Exception(f"Invalid file type: {file.suffix}")

In [20]:
data = read_file("total_output.db")
data.drop(["JoinDate"], axis=1, inplace=True)
data.Age = data.Age.astype(int)
data

,index,ID,Name,Age,City,Salary
0,0,1,Name_1,20,City_0,104509.45
1,1,2,Name_2,21,City_1,53401.42
2,2,3,Name_3,22,City_2,118291.15
3,3,4,Name_0,23,City_3,60119.26
4,4,5,Name_5,24,City_4,111892.54
...,...,...,...,...,...,...
2995,2995,2996,Name_2996,65,City_95,96917.54
2996,2996,2997,Name_2997,66,City_96,76070.36
2997,2997,2998,Name_2998,67,City_97,49177.59
2998,2998,2999,Name_2999,68,City_98,109764.15


### Basic statistics from data

In [124]:
print(f"Mean salary in data is {data.Salary.mean()}")
print(f"Mean age in data is {data.Age.mean()}")
print(f"Most people come from {data.City.mode()[0]}")


Mean salary in data is 74032.20057692309
Mean age in data is 44.49166666666667
Most people come from City_101


### Number of people per age group

In [125]:
px.histogram(data, x='Age', nbins=10, log_y=True, title="Number of people per age group").update_layout(bargap=0.01).show()

### Salary by decade

In [154]:
px.histogram(data, x='Age', y="Salary", nbins=10, histfunc='avg', log_y=True, title="Average Salary by Age Group").update_layout(bargap=0.01).show()

### Greatest salary by Age group

In [127]:
px.histogram(data, x='Age', y="Salary", nbins=10, histfunc='max', log_y=True, title="Largest salary by Age Group").update_layout(bargap=0.01).show()

### Top 10 most populated cities

In [160]:
px.histogram(data.City.value_counts().reset_index().head(10), x='City',y='count', nbins=10, title="Top 10 most populated cities").show()

### Distribution of Salaries

In [129]:
px.histogram(data, x='Salary', nbins=10, log_y=True, title="Distributions of salary").update_layout(bargap=0.01).show()

### Percentage of salaries above 100k

In [130]:
px.pie(data.Salary.apply(lambda x: f"{'More' if x >= 100000 else 'Less'} than 100k"), names='Salary', labels=['hello'], title='Percentage of salaries above 100K', width=700)

### Top 10 cities with highest average salaries

In [131]:
px.histogram(data.sort_values(by="Salary", ascending=False).head(10), x='City', y="Salary", nbins=10, histfunc='avg', log_y=True, title="Top 10 cities with highest average salaries").show()

In [132]:
px.histogram(data.sort_values(by="Salary", ascending=True).head(10), x='City', y="Salary", nbins=10, histfunc='avg', log_y=True, title="Top 10 cities with lowest average salaries").show()

### Average age in top 10 cities with highest average salaries

In [133]:
px.histogram(data.sort_values(by="Salary", ascending=False).head(10), x='City', y="Age", nbins=10, histfunc='avg', title="Average age in top 10 cities with highest average salaries").show()

### Average age in top 10 cities with lowest average salaries

In [207]:
px.histogram(data.sort_values(by="Salary", ascending=True).head(10), x='City', y="Age", nbins=10, histfunc='avg', title="Average age in top 10 cities with lowest average salaries").show()

### Salary-Age Correlation

In [152]:
salary_by_age = data.groupby('Age').Salary.mean().reset_index()
px.scatter(salary_by_age, x='Age', y='Salary', trendline='ols', trendline_color_override='red')

In [324]:
import attr
import plotly.graph_objs as go

@attr.s
class ReportGenerator:
    """Generates different plotly figures based on data"""

    data = attr.ib(type=pd.DataFrame)

    def histogram(self, x_axis: str, y_axis: str = 'count', histfunc: str = 'avg', nbins: int = 10, top_most: int = None) -> go.Figure:
        args = self.data[x_axis].value_counts().reset_index() if y_axis == 'count' else self.data
        kwargs = {'x': x_axis, 
                  'y': y_axis, 
                  'log_y': True, 
                  'histfunc': histfunc, 
                  'nbins': nbins
                  }
        
        if top_most and args[x_axis].dtype in ['category', 'object']:
            args = args.sort_values(by=y_axis, ascending= top_most<=0).head(int(np.sqrt(top_most**2)))
        
        return px.histogram(args, **kwargs).update_layout(bargap=0.02)
    

In [325]:
r = ReportGenerator(data)
r.histogram(x_axis='Age', top_most=10).show()
r.histogram(x_axis='Age', nbins=10).show()
r.histogram(x_axis='Age', y_axis='Salary', nbins=10).show()
r.histogram(x_axis='City', y_axis='Salary', top_most=10).show()
r.histogram(x_axis='City', y_axis='Age').show()